In [6]:
import geopandas as gpd
from geopandas.geoseries import *
import numpy as np

# 190 PLZs
plz = gpd.read_file('plz.geojson')

plr = gpd.read_file('output_berlin-lor.planungsraeume.geojson')

plr_data = pd.read_csv('LOR_Planungsraeume_2019.csv', sep=';')
plr_data.rename(columns={'Planungsraum': 'PLANUNGSRAUM', 'Bezirksregion': 'BEZIRKSREGION'}, inplace=True)

plr = plr.merge(plr_data, on=['PLANUNGSRAUM', 'BEZIRKSREGION'])

df = pd.DataFrame(columns = ['PLR', 'PLR_NAME', 'PLZ'])
df['PLZ'] = np.empty((len(plr), 0)).tolist()
df.head()
for indexPLR, rowPLR in plr.iterrows():
    df.loc[indexPLR, 'PLR'] = rowPLR.Schlüssel
    df.loc[indexPLR, 'PLR_NAME'] = rowPLR.PLANUNGSRAUM
    for indexPLZ, rowPLZ in plz.iterrows():
        if rowPLZ.geometry.intersects(rowPLR.geometry):
            df.loc[indexPLR, 'PLZ'].append(str(rowPLZ.spatial_name))

df.to_csv('PLR2PLZ.csv')


df2 = pd.DataFrame(columns = ['PLZ', 'PLR'])
df2['PLR'] = np.empty((len(plz), 0)).tolist()

for indexPLZ, rowPLZ in plz.iterrows():
    df2.loc[indexPLZ, 'PLZ'] = rowPLZ.spatial_name
    for indexPLR, rowPLR in plr.iterrows():
        if rowPLR.geometry.intersects(rowPLZ.geometry):
            df2.loc[indexPLZ, 'PLR'].append(rowPLR.Schlüssel)

df2.to_csv('PLZ2PLR.csv')
